# Silver - tb_criticidade (SRC)

**Catálogo**: v_credit  
**Schema**: silver  
**Tabela**: tb_criticidade

**Objetivo**: Popular tabela de domínio de criticidades

**Tipo**: Lookup Table (valores fixos)

**Pré-requisito**: Tabela silver.tb_criticidade já criada via DDL

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import datetime

processing_timestamp = spark.sql("SELECT current_timestamp() as ts").collect()[0]['ts']
print(f"Timestamp do processamento: {processing_timestamp}")


## 1. Configuração

In [0]:
TABLE_TARGET = "v_credit.silver.tb_criticidade"

print(f"Tabela Destino: {TABLE_TARGET}")

## 2. Definir Dados de Criticidade

**Valores Fixos**:
- 1: Baixa
- 2: Média
- 3: Alta

In [0]:
# Dados das criticidades (valores fixos)
criticidades_data = [
    (1, "Baixa"),
    (2, "Média"),
    (3, "Alta")
]

# Schema
schema = StructType([
    StructField("cd_criticidade", LongType(), False),
    StructField("ds_criticidade", StringType(), False)
])

# Criar DataFrame
df_criticidades = spark.createDataFrame(criticidades_data, schema)

print(f"Total de criticidades: {df_criticidades.count()}")

In [0]:
# Visualizar dados
print("Criticidades a serem inseridas:")
display(df_criticidades)

## 3. Adicionar Metadados

In [0]:
# Usar o timestamp capturado no início (garante consistência)
lit_timestamp = F.lit(processing_timestamp)

# Adicionar metadados
df_silver = (df_criticidades
    .withColumn("dt_ingestion", lit_timestamp)
    .withColumn("dc_origem", F.lit("sistema"))
)

print(f"✅ Metadados adicionados")

In [0]:
# Schema final
print("Schema final:")
df_silver.printSchema()

In [0]:
# Visualizar dados finais
print("Dados finais a serem inseridos:")
display(df_silver)

## 4. MERGE na Silver

**Estratégia**: 
- Para lookup tables com valores fixos, podemos usar MERGE também
- Garante que não duplica se rodar múltiplas vezes

In [0]:
# Criar temp view
df_silver.createOrReplaceTempView("temp_criticidade_updates")

print(f"Executando MERGE em: {TABLE_TARGET}")
print(f"Total de registros: {df_silver.count()}")
print("="*80)

In [0]:
# MERGE SQL
spark.sql(f"""
    MERGE INTO {TABLE_TARGET} AS target
    USING temp_criticidade_updates AS source
    ON target.cd_criticidade = source.cd_criticidade
    
    WHEN MATCHED THEN
        UPDATE SET
            target.ds_criticidade = source.ds_criticidade,
            target.dt_ingestion = source.dt_ingestion,
            target.dc_origem = source.dc_origem
    
    WHEN NOT MATCHED THEN
        INSERT (
            cd_criticidade,
            ds_criticidade,
            dt_ingestion,
            dc_origem
        )
        VALUES (
            source.cd_criticidade,
            source.ds_criticidade,
            source.dt_ingestion,
            source.dc_origem
        )
""")

print("✅ MERGE concluído com sucesso!")

## 5. Verificação

In [0]:
# Ler tabela final
df_final = spark.table(TABLE_TARGET)

print("="*80)
print("VERIFICAÇÃO DA TABELA")
print("="*80)
print(f"Tabela: {TABLE_TARGET}")
print(f"Total de registros: {df_final.count()}")

In [0]:
# Mostrar dados
print("\nDados na tabela:")
display(df_final.orderBy("cd_criticidade"))

## 6. Validações

In [0]:
print("VALIDAÇÕES:")
print("="*80)

# Verificar se tem exatamente 3 registros
total = df_final.count()
if total != 3:
    print(f"⚠️  AVISO: Esperado 3 registros, encontrado {total}")
else:
    print(f"✅ Total correto: {total} registros")

# Verificar valores esperados
valores_esperados = ["Baixa", "Média", "Alta"]
valores_encontrados = [row['ds_criticidade'] for row in df_final.select("ds_criticidade").collect()]

if set(valores_esperados) == set(valores_encontrados):
    print("✅ Todos os valores esperados estão presentes")
else:
    print(f"⚠️  Valores esperados: {valores_esperados}")
    print(f"   Valores encontrados: {valores_encontrados}")

## 7. Uso com tb_motivo

In [0]:
# Exemplo de JOIN com tb_motivo
print("Exemplo de uso com tb_motivo:")
print("="*80)

try:
    df_join = spark.sql("""
        SELECT 
            m.cd_motivo,
            m.ds_motivo,
            m.ds_criticidade,
            c.cd_criticidade
        FROM v_credit.silver.tb_motivo m
        LEFT JOIN v_credit.silver.tb_criticidade c
            ON m.ds_criticidade = c.ds_criticidade
        LIMIT 5
    """)
    
    display(df_join)
    
    # Verificar se há registros sem match
    sem_match = spark.sql("""
        SELECT COUNT(*) as total
        FROM v_credit.silver.tb_motivo m
        LEFT JOIN v_credit.silver.tb_criticidade c
            ON m.ds_criticidade = c.ds_criticidade
        WHERE c.cd_criticidade IS NULL
    """).collect()[0]['total']
    
    if sem_match > 0:
        print(f"\n⚠️  AVISO: {sem_match} motivos sem criticidade válida!")
    else:
        print("\n✅ Todos os motivos têm criticidade válida")
        
except Exception as e:
    print(f"⚠️  tb_motivo ainda não existe ou não tem dados: {e}")

## 8. Histórico Delta Lake

In [0]:
print("HISTÓRICO DE VERSÕES:")
print("="*80)

display(spark.sql(f"DESCRIBE HISTORY {TABLE_TARGET} LIMIT 5"))

## ✅ tb_criticidade Populada!

**Tabela**: `v_credit.silver.tb_criticidade`

**Registros**:
- 3 níveis de criticidade
  - 1: Baixa
  - 2: Média
  - 3: Alta

**Uso**:
```sql
SELECT 
    m.ds_motivo,
    m.ds_criticidade,
    c.cd_criticidade
FROM silver.tb_motivo m
JOIN silver.tb_criticidade c 
    ON m.ds_criticidade = c.ds_criticidade
```

**Observação**: 
- Join é feito por `ds_criticidade` (texto)
- Permite adicionar `cd_criticidade` (código numérico) nas análises
- Facilita agrupamentos e ordenações

In [0]:
# Log final
print(f"\n{'='*80}")
print(f"✅ Processamento concluído com sucesso!")
print(f"Timestamp utilizado em todos os registros: {processing_timestamp}")
print(f"Tabela: {TABLE_TARGET}")
print(f"Registros: {spark.table(TABLE_TARGET).count()}")
print(f"{'='*80}")